In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np 
import time
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import models, datasets, transforms
import torch.optim as optim
from PIL import Image
import os 
import copy
import glob


In [2]:
object_classes = ['buoys', 'harbour', 'human', 'large_commercial_vessel', 'leisure_craft', 'sailboats', 'small_medium_fishing_boat']
num_classes = len(object_classes)

data_path = r'C:\DTU\master_thesis\fsl\Object-classification-with-few-shot-learning\data\raw\updated_ds\sailboats'

model_name = "resnet"


In [3]:
#Remove small images with one of the dimensions lesser than 16 pixels

for object_class in object_classes:
    num_deleted_images = 0
    data_path = os.path.join('C:\\DTU\\master_thesis\\fsl\\Object-classification-with-few-shot-learning\\data\\raw\\updated_ds', object_class)
    for (root, dirs, files) in os.walk(data_path, topdown = True):
        for file in files:
            file_path = os.path.join(data_path, file)
            with Image.open(file_path) as img:
                img_size = img.size
            if img_size[0] < 16 or img_size[1] < 16:
                os.remove(file_path)
                num_deleted_images += 1
    print(f'The number of {object_class} images deleted is: ', num_deleted_images)     


The number of buoys images deleted is:  0
The number of harbour images deleted is:  0
The number of human images deleted is:  0
The number of large_commercial_vessel images deleted is:  0
The number of leisure_craft images deleted is:  0
The number of sailboats images deleted is:  0
The number of small_medium_fishing_boat images deleted is:  0


In [4]:
batch_size = 8

# Number of epochs to train for
num_epochs = 10

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [5]:
class MaritimeDataset(Dataset):

    def __init__(self, root_dir="", transform=transforms) -> None:
        self.dataset = datasets.ImageFolder(root=root_dir)
        self.transform = transform
        self.paths, self.labels = zip(*self.dataset.samples)
        self.classes = self.dataset.class_to_idx

    def __len__(self):
        return len(self.labels)


    def __getitem__(self, index):

        path, target = self.paths[index], self.labels[index]

        with Image.open(path) as img:

            if self.transform is not None:
                img = self.transform(img)

        return img, target, index, path




In [6]:
image_size = 128

data_dir = r'C:\DTU\master_thesis\fsl\Object-classification-with-few-shot-learning\data\raw\updated_ds'

complete_dataset = MaritimeDataset(root_dir = data_dir,
                                    transform = transforms.Compose([
                                        transforms.Resize(image_size),
                                        transforms.CenterCrop(image_size),
                                        transforms.ToTensor(),
                                        transforms.RandomHorizontalFlip()
                                    ]))

In [7]:
#Splittin the data into train and test sets

train_length = int(0.7* len(complete_dataset))

test_length = len(complete_dataset) - train_length

train_dataset, test_dataset = torch.utils.data.random_split(complete_dataset, (train_length, test_length))

In [8]:
from tqdm import tqdm

def mean_and_std():
    """This function calculates the mean and standard deviation of the dataset"""

    dataloader = DataLoader(complete_dataset, shuffle=False, batch_size=12)
    mean = 0.0
    std = 0.0
    nb_samples = 0.0

    for images, _, _, _ in tqdm(dataloader):
        batch_samples = images.size(0)
        data = images.view(batch_samples, images.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        nb_samples += batch_samples

    mean /= nb_samples
    std /= nb_samples
    return mean, std

mean, std = mean_and_std()

print(f'The mean of the dataset is', mean)
print(f'The standard deviation of the dataset is', std)

100%|██████████| 1504/1504 [00:27<00:00, 53.97it/s]

The mean of the dataset is tensor([0.4609, 0.4467, 0.4413])
The standard deviation of the dataset is tensor([0.1314, 0.1239, 0.1198])


In [9]:
train_dataset

In [20]:
workers = 6
train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                            shuffle=True, num_workers=workers)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size,
                            shuffle=True, num_workers=workers)

In [11]:
n_classes = len(complete_dataset.classes.keys())

In [12]:
import torch.optim.lr_scheduler

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

False


In [14]:
device

device(type='cpu')

In [15]:
model = models.resnet18(pretrained=True)

In [16]:
opt = torch.optim.Adam(model.parameters(), lr=1e-5)

loss = torch.nn.CrossEntropyLoss()

criterion = nn.NLLLoss()

In [17]:
# epochs = 1
# steps = 0
# running_loss = 0
# print_every = 10
# train_losses, test_losses = [], []
# for epoch in tqdm(range(epochs)):
#     for inputs, labels in train_dataloader:
#         steps += 1
#         inputs, labels = inputs.to(device), labels.to(device)
#         opt.zero_grad()
#         logps = model.forward(inputs)
#         loss = criterion(logps, labels)
#         loss.backward()
#         opt.step()
#         running_loss += loss.item()
        
#         if steps % print_every == 0:
#             test_loss = 0
#             accuracy = 0
#             model.eval()
#             with torch.no_grad():
#                 for inputs, labels in tqdm(test_dataloader):
#                     inputs, labels = inputs.to(device), labels.to(device)
#                     logps = model.forward(inputs)
#                     batch_loss = criterion(logps, labels)
#                     test_loss += batch_loss.item()
                    
#                     ps = torch.exp(logps)
#                     top_p, top_class = ps.topk(1, dim=1)
#                     equals = top_class == labels.view(*top_class.shape)
#                     accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
#             train_losses.append(running_loss/len(train_dataloader))
#             test_losses.append(test_loss/len(test_dataloader))                    
#             print(f"Epoch {epoch+1}/{epochs}.. "
#                   f"Train loss: {running_loss/print_every:.3f}.. "
#                   f"Test loss: {test_loss/len(test_dataloader):.3f}.. "
#                   f"Test accuracy: {accuracy/len(test_dataloader):.3f}")
#             running_loss = 0
#             model.train()
# torch.save(model, 'maritimemodelv1.pth')

In [18]:

# convolutional_network = models.resnet18(pretrained=False)
# convolutional_network.fc = nn.Flatten()


# import matplotlib.pyplot as plt



# for param in convolutional_network.parameters():
#     param.requires_grad = True
# device = torch.device("cuda")
# print('1')
# def train(model, train_dataloader, criterion, optimizer, epochs = 5):
#     print('2')
#     train_loss =[]
#     for e in tqdm(range(epochs)):
#         print('3')
#         running_loss =0
#         for images, labels in tqdm(train_dataloader):
#             print('4')
#             inputs, labels = images.to(device), labels.to(device)

#             optimizer.zero_grad()
#             img = model(inputs)
            
#             loss = criterion(img, labels)
#             running_loss+=loss
#             loss.backward()
#             optimizer.step()
#         print("Epoch : {}/{}..".format(e+1,epochs),
#          "Training Loss: {:.6f}".format(running_loss/len(train_dataloader))) 
#         train_loss.append(running_loss)
#     #plt.plot(train_loss,label="Training Loss")
#     #plt.show() 
#     filename_pth = 'model_resnet18_fsl.pth'
#     torch.save(model.state_dict(), filename_pth)

# def run():
#     torch.multiprocessing.freeze_support()
#     print('loop')

# epochs = 5
# convolutional_network.train()
# optimizer = optim.Adam(convolutional_network.parameters(), lr=0.001)
# criterion = nn.CrossEntropyLoss()    
# train(convolutional_network,train_dataloader,criterion, optimizer, epochs) 

In [21]:
print(len(train_dataloader.dataset))

12628


In [22]:
for i in tqdm(train_dataloader.dataset):
    print(i)

  0%|          | 0/1579 [00:00<?, ?it/s]